In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import copy
from pathlib import Path
import torch

In [ ]:
from src.training.SmallRawDatasetNumpy import SmallRawDatasetNumpy
from src.training.load_config import load_config


In [ ]:
run_config = load_config()
dataset_path = Path(run_config['cropped_raw_subdir'])
align_csv = dataset_path / run_config['secondary_align_csv']
device=run_config['device']

batch_size = run_config['batch_size']
lr = run_config['lr_base'] * batch_size
clipping =  run_config['clipping']

num_epochs = run_config['num_epochs_pretraining']
val_split = run_config['val_split']
crop_size = run_config['crop_size']
experiment = run_config['mlflow_experiment']
model_params = run_config['model_params']
rggb = model_params['rggb']

In [ ]:
dataset = SmallRawDatasetNumpy(dataset_path, align_csv, crop_size=crop_size, validation=True)


In [ ]:
# with mlflow.start_run(run_id=existing_run_id) as run:
#     print(f"Re-opened run: {run.info.run_name}")
    
#     # Log your new validation metric
#     mlflow.log_metric("final_validation_accuracy", new_validation_metric)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import correlate2d
from scipy.ndimage import gaussian_filter
from skimage.metrics import structural_similarity as ssim

# --- Metric functions ---

def structure_score_autocorr(img_np):
    img = img_np - img_np.mean()
    corr = correlate2d(img, img, mode='full')
    corr /= corr.max() + 1e-8
    center = np.array(corr.shape) // 2
    corr[center[0], center[1]] = 0
    return float(np.mean(corr))

def fourier_entropy(img_np):
    f = np.fft.fftshift(np.fft.fft2(img_np))
    mag = np.abs(f)
    mag = mag / (mag.sum() + 1e-8)
    return float(-np.sum(mag * np.log(mag + 1e-12)))

def laplacian_variance(img_np):
    import cv2
    lap = cv2.Laplacian(img_np, cv2.CV_64F)
    return float(lap.var())

def self_ssim(img_np, sigma=1.0):
    blurred = gaussian_filter(img_np, sigma)
    return float(ssim(img_np, blurred, data_range=img_np.max() - img_np.min()))

# --- Visualization and analysis function ---

def analyze_dataset_structure(dataset, indices, out_dir="structure_analysis"):
    import os
    os.makedirs(out_dir, exist_ok=True)
    records = []

    for idx in indices:
        row = dataset[idx]
        noisy = row['noisy']
        gt = row['aligned']

        # Ensure CPU numpy grayscale (if multi-channel)
        def to_gray(x):
            if isinstance(x, torch.Tensor):
                x = x.detach().cpu().numpy()
            if x.ndim == 3 and x.shape[0] in [1,3]:
                x = np.moveaxis(x, 0, -1)
            if x.ndim == 3:
                x = np.mean(x, axis=-1)
            return np.clip(x, 0, 1)

        noisy_np = to_gray(noisy)
        gt_np = to_gray(gt)
        diff_np = np.abs(noisy_np - gt_np)

        # Compute metrics
        metrics = {
            "idx": idx,
            "fourier_entropy_noisy": fourier_entropy(noisy_np),
            "lap_var_noisy": laplacian_variance(noisy_np),
            "self_ssim_noisy": self_ssim(noisy_np),
            "autocorr_noisy": structure_score_autocorr(noisy_np),
            "fourier_entropy_diff": fourier_entropy(diff_np),
            "lap_var_diff": laplacian_variance(diff_np),
        }
        records.append(metrics)

        # Plot
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        for ax, im, title in zip(axes, [noisy_np, gt_np, diff_np],
                                 ['Noisy', 'Aligned', 'Difference']):
            ax.imshow(im, cmap='gray')
            ax.set_title(title)
            ax.axis('off')

        txt = "\n".join([
            f"Entropy: {metrics['fourier_entropy_noisy']:.3f}",
            f"LapVar: {metrics['lap_var_noisy']:.3f}",
            f"SelfSSIM: {metrics['self_ssim_noisy']:.3f}",
            f"AutoCorr: {metrics['autocorr_noisy']:.3f}",
        ])
        plt.gcf().text(0.75, 0.5, txt, fontsize=10, va='center')
        plt.tight_layout()
        plt.savefig(f"{out_dir}/example_{idx:04d}.png", dpi=150)
        plt.close(fig)

    df = pd.DataFrame.from_records(records)
    df.to_csv(f"{out_dir}/structure_metrics.csv", index=False)
    print(f"Saved {len(df)} examples and metrics to {out_dir}")
    return df


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import correlate2d
from scipy.ndimage import gaussian_filter
from skimage.metrics import structural_similarity as ssim

def analyze_dataset_structure(dataset, indices, out_dir="structure_analysis"):
    import os
    os.makedirs(out_dir, exist_ok=True)
    records = []

    for idx in indices:
        row = dataset[idx]
        noisy = row['noisy'].permute(1, 2, 0)**(1/2.2)
        gt = row['aligned'].permute(1, 2, 0)**(1/2.2)
        diff = noisy-gt +0.5
        # Plot
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        for ax, im, title in zip(axes, [noisy, gt, diff],
                                 ['Noisy', 'Aligned', 'Difference']):
            ax.imshow(im)
            ax.set_title(title)
            ax.axis('off')

        plt.tight_layout()
        plt.show()
        plt.close(fig)

In [ ]:
import numpy as np
mask = dataset.df.iso.values == 400
matching_indices_in_subset = np.nonzero(mask)[0]
matching_indices_in_subset

In [ ]:
analyze_dataset_structure(dataset, matching_indices_in_subset,out_dir="structure_analysis_no_align")
